### Define jobname

In [1]:
pdbid = 'dsdna'
drug  = None
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid
indexfile=f'{jobname}.simulate.ndx'
mdp_folder = "../../../smsl/opt/mdp"

### make a centered reference structure

In [2]:
def rmpbc(filename,indexfile='index.ndx',center=0,output=0, postfix='c'):
    file = ".".join(filename.split('.')[:-1]).split('_')[0]
    extend = filename.split('.')[-1]
    cmd1 = f'echo "{center} {output}" | gmx trjconv -s {file}.tpr -f {filename} -o {file}_{postfix}.{extend} -pbc res -center -boxcenter zero -n {indexfile} -ndec 5 -ur compact'
    print("##========================command=line========================##")
    print(cmd1,sep="\n")
    print("##======================end=command=line======================##")
    !{cmd1}

In [ ]:
def grep_and_fit(filename,indexfile='index.ndx',center=0,output=0, postfix='rms'):
    file = ".".join(filename.split('.')[:-1]).split('_')[0]
    extend = filename.split('.')[-1]
    cmd1 = f'echo "{center} {output}" | gmx trjconv -s rms.tpr -f {filename} -o {file}_{postfix}.{extend} -n {indexfile} -fit rot+trans -ndec 5'
    print("##========================command=line========================##")
    print(cmd1,sep="\n")
    print("##======================end=command=line======================##") 
    !{cmd1}

In [3]:
rmpbc("npt2.pdb", indexfile, center='ChainA')

##========================command=line========================##
echo "DNA 0" | gmx trjconv -s npt2.tpr -f npt2.pdb -o npt2_c.pdb -pbc res -center -boxcenter zero -n 2jpz.simulate.ndx -ndec 5 -ur compact
##======================end=command=line======================##
                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx trjconv -s npt2.tpr -f npt2.pdb -o npt2_c.pdb -pbc res -center -boxcenter zero -n 2jpz.simulate.ndx -ndec 5 -ur compact

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write pdb: Protein data bank file
Reading file npt2.tpr, VERSION 2023.2 (single precision)
Reading file npt2.tpr, VERSION 2023.2 (single precision)
Select group for centering
Group     0 (         System) has 12201 elements
Group     1 (         

### make a tpr using centered reference structure

In [4]:
!gmx grompp -f {mdp_folder}/min2.mdp -c npt2_c.pdb -r npt2_c.pdb -p topol.top -o rms.tpr

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx grompp -f ../../../smsl/opt/mdp/min2.mdp -c npt2_c.pdb -r npt2_c.pdb -p topol.top -o rms.tpr


NOTE 1 [file ../../../smsl/opt/mdp/min2.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to -134692930

Generated 3828 of the 3828 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 3828 of the 3828 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'K'

turn

### Center protein for a solvated trajectory and put residues in the unit cell 
- Input and output are both solvated trajectories
- 1. make residues in the unit cell
- The sequence of commands here are used to generate the "process.solvated.traj" shell scripts for production runs

In [5]:
rmpbc("heat.xtc", indexfile, center='ChainA')
rmpbc("nvt1.xtc", indexfile, center='ChainA')
rmpbc("npt1.xtc", indexfile, center='ChainA')
rmpbc("npt2.xtc", indexfile, center='ChainA')

##========================command=line========================##
echo "DNA 0" | gmx trjconv -s heat.tpr -f heat.xtc -o heat_c.xtc -pbc res -center -boxcenter zero -n 2jpz.simulate.ndx -ndec 5 -ur compact
##======================end=command=line======================##
                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx trjconv -s heat.tpr -f heat.xtc -o heat_c.xtc -pbc res -center -boxcenter zero -n 2jpz.simulate.ndx -ndec 5 -ur compact

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file heat.tpr, VERSION 2023.2 (single precision)
Reading file heat.tpr, VERSION 2023.2 (single precision)
Select group for centering
Group     0 (         System) has 12201 eleme

### get a no-solvent reference structure

In [22]:
!echo "Model"     | gmx editconf -f npt2_c.pdb -o npt2_rms.pdb -n {jobname}.simulate.ndx
!echo "non-Water" | gmx editconf -f npt2_c.pdb -o npt2_noh.pdb -n {jobname}.simulate.ndx

                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx editconf -f npt2_c.pdb -o npt2_rms.pdb -n 2jpz.simulate.ndx

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 12201 atoms
Volume: 122.168 nm^3, corresponds to roughly 54900 electrons
No velocities found

Select a group for output:
Group     0 (         System) has 12201 elements
Group     1 (            DNA) has   715 elements
Group     2 (              K) has    31 elements
Group     3 (             CL) has    10 elements
Group     4 (            Ion) has    41 elements
Group     5 (          Water) has 11445 elements
Group     6 (            SOL) has 11445 elements
Group     7 (      non-Water) has   756 elements
Group     8 ( Water_and_ions) has 11486 elements
Group     9

### Output non-water parts with best fit 
- The index file needs to be setup first
- The sequence of commands here are used to generate the "rms.traj" shell scripts for production runs

In [8]:
grep_and_fit("heat_c.xtc", indexfile, center='ChainA', output='Model')
grep_and_fit("nvt1_c.xtc", indexfile, center='ChainA', output='Model')
grep_and_fit("npt1_c.xtc", indexfile, center='ChainA', output='Model')
grep_and_fit("npt2_c.xtc", indexfile, center='ChainA', output='Model')

##========================command=line========================##
echo "DNA Model" | gmx trjconv -s rms.tpr -f heat_c.xtc -o heat_rms.xtc -n 2jpz.simulate.ndx -fit rot+trans -ndec 5
##======================end=command=line======================##
                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx trjconv -s rms.tpr -f heat_c.xtc -o heat_rms.xtc -n 2jpz.simulate.ndx -fit rot+trans -ndec 5

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file rms.tpr, VERSION 2023.2 (single precision)
Reading file rms.tpr, VERSION 2023.2 (single precision)
Select group for least squares fit
Group     0 (         System) has 12201 elements
Group     1 (            DNA) has   715 

In [9]:
grep_and_fit("heat_c.xtc", indexfile, center='ChainA', output='non-Water', postfix='noh')
grep_and_fit("nvt1_c.xtc", indexfile, center='ChainA', output='non-Water', postfix='noh')
grep_and_fit("npt1_c.xtc", indexfile, center='ChainA', output='non-Water', postfix='noh')
grep_and_fit("npt2_c.xtc", indexfile, center='ChainA', output='non-Water', postfix='noh')

##========================command=line========================##
echo "DNA non-Water" | gmx trjconv -s rms.tpr -f heat_c.xtc -o heat_noh.xtc -n 2jpz.simulate.ndx -fit rot+trans -ndec 5
##======================end=command=line======================##
                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx trjconv -s rms.tpr -f heat_c.xtc -o heat_noh.xtc -n 2jpz.simulate.ndx -fit rot+trans -ndec 5

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file rms.tpr, VERSION 2023.2 (single precision)
Reading file rms.tpr, VERSION 2023.2 (single precision)
Select group for least squares fit
Group     0 (         System) has 12201 elements
Group     1 (            DNA) has   

### Cat rms trajectories
- Use cat.traj to combine production run trajectories

In [17]:
cmd = '''\
c
c
c
c
'''

!echo -e "{cmd}" | gmx trjcat -f heat_c.xtc nvt1_c.xtc npt1_c.xtc npt2_c.xtc -o equilibrium_c.xtc -settime

                      :-) GROMACS - gmx trjcat, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx trjcat -f heat_c.xtc nvt1_c.xtc npt1_c.xtc npt2_c.xtc -o equilibrium_c.xtc -settime

Reading frame       1 time   10.000   


Enter the new start time (ps) for each file.
There are two special options, both disable sorting:

c (continue) - The start time is taken from the end
of the previous file. Use it when your continuation run
restarts with t=0.

l (last) - The time in this file will be changed the
same amount as in the previous. Use it when the time in the
new run continues from the end of the previous one,
since this takes possible overlap into account.

          File             Current start (ps)  New start (ps)
---------------------------------------------------------
               heat_c.xtc        0.000 ps      

In [18]:
cmd = '''\
c
c
c
c
'''

!echo -e "{cmd}" | gmx trjcat -f heat_noh.xtc nvt1_noh.xtc npt1_noh.xtc npt2_noh.xtc -o equilibrium_noh.xtc -settime

                      :-) GROMACS - gmx trjcat, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx trjcat -f heat_noh.xtc nvt1_noh.xtc npt1_noh.xtc npt2_noh.xtc -o equilibrium_noh.xtc -settime

Reading frame       1 time   10.000   


Enter the new start time (ps) for each file.
There are two special options, both disable sorting:

c (continue) - The start time is taken from the end
of the previous file. Use it when your continuation run
restarts with t=0.

l (last) - The time in this file will be changed the
same amount as in the previous. Use it when the time in the
new run continues from the end of the previous one,
since this takes possible overlap into account.

          File             Current start (ps)  New start (ps)
---------------------------------------------------------
             heat_noh.xtc        0.00

In [19]:
cmd = '''\
c
c
c
c
'''

!echo -e "{cmd}" | gmx trjcat -f heat_rms.xtc nvt1_rms.xtc npt1_rms.xtc npt2_rms.xtc -o equilibrium_rms.xtc -settime

                      :-) GROMACS - gmx trjcat, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii/c.gromacs.run_md/1.create_system
Command line:
  gmx trjcat -f heat_rms.xtc nvt1_rms.xtc npt1_rms.xtc npt2_rms.xtc -o equilibrium_rms.xtc -settime

Reading frame       1 time   10.000   


Enter the new start time (ps) for each file.
There are two special options, both disable sorting:

c (continue) - The start time is taken from the end
of the previous file. Use it when your continuation run
restarts with t=0.

l (last) - The time in this file will be changed the
same amount as in the previous. Use it when the time in the
new run continues from the end of the previous one,
since this takes possible overlap into account.

          File             Current start (ps)  New start (ps)
---------------------------------------------------------
             heat_rms.xtc        0.00

## Visualize
```bash
pymol rms.pdb heat_rms.xtc  
vmd -gro rms.pdb heat_rms.xtc  
```

### Move data to next folder

In [20]:
!cp {indexfile} rms.tpr           ../2.simulation  ## for rmpbc
!cp npt2.pdb npt2.cpt *.top *.itp ../2.simulation  ## for mdrun

In [24]:
!cp npt2_rms.pdb                  ../3.trajectory_data/{jobname}.npt2.pdb